# TSA Claim Reimbursement Predictor

By: Israel Fernandez, Haider Syed

Objective: The objective of this report is to write a machine learning algorithm to answer some crucial questions that people may have when taking flights with valuable belongings. We will illustrate the likelyhood of getting a refund, what percentage of what you ask for you might actually get reimbursed, and whether or not you should feel comfortable taking said belongings to the airport.

In [5]:
# All imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import tree
from IPython.display import display, HTML

We took data from https://www.kaggle.com/datasets/terminal-security-agency/tsa-claims-database/data

In [6]:
df = pd.read_csv('tsa_claims.csv')

C:\Users\israe\AppData\Local\Temp\ipykernel_26664\3021715513.py:1: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('tsa_claims.csv')


The dataset contains information about claims that were made against the TSA for property damage, lost or damaged property. These claims are generally filed against the TSA for personal injuries, lost or damaged property during screenings. 

This dataset is the record of claims between 2002 - 2015 recoreded by the TSA.

The dataset contains the following data:

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204267 entries, 0 to 204266
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Claim Number   204267 non-null  object
 1   Date Received  204004 non-null  object
 2   Incident Date  202084 non-null  object
 3   Airport Code   195743 non-null  object
 4   Airport Name   195743 non-null  object
 5   Airline Name   169893 non-null  object
 6   Claim Type     196354 non-null  object
 7   Claim Site     203527 non-null  object
 8   Item           200301 non-null  object
 9   Claim Amount   200224 non-null  object
 10  Status         204262 non-null  object
 11  Close Amount   135315 non-null  object
 12  Disposition    131359 non-null  object
dtypes: object(13)
memory usage: 20.3+ MB


The data contains a lot of information that, although is good to have, is not necessary for our purposes. We are going to clean it up a bit.

In [ ]:
# Helper function to clean data
def clean_data(col):
    cleaned_data = df[col].str.replace("$", "", regex=False).str.replace(";","").str.replace(" ", "").replace("-",float("NaN"))
    return cleaned_data.convert_dtypes(float)
# Clean the claim amounts to floats
df["Claim Amount"] = clean_data('Claim Amount')

# Drop NA values for Close Amount and Claim Amount as they are not important
df = df.dropna(subset = ['Close Amount'])
df = df.dropna(subset = ['Claim Amount'])

# We do not need Airline Names. We will be using Airline Codes
df = df.drop('Airline Name', axis=1)

# Ignore Items that do not have a description
df = df.dropna(subset = ['Item'])

# 
df['Item'].value_counts()
item_counts = df['Item'].value_counts()
items_to_keep = item_counts[item_counts >= 1000].index
df = df[df['Item'].isin(items_to_keep)]
df = df[df['Item'] != 'Other']
